In [ ]:
from geopy.geocoders import Nominatim
import csv
import time
import pandas as pd
import os
import logging

In [ ]:
df1 = pd.read_csv('checkpoint.csv')
df2 = pd.read_csv('fuel-prices-for-be-assessment.csv')



In [18]:
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(address)
    
    if location:
        return location.raw['osm_id']
    else:
        return None

logging.basicConfig(
    filename='geocode.log', 
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def geocode_csv(input_file, output_file, checkpoint_file='checkpoint.csv', sleep_interval=1):
    
    # Initialize logging
    logging.basicConfig(
        filename='geocode.log', 
        level=logging.INFO, 
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Initialize cache for City, State to osm_id mapping
    city_state_osmid_mapping = {}
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                city_state = (row['City'], row['State'])
                osm_id = row.get('osm_id', '')
                city_state_osmid_mapping[city_state] = osm_id

    # Open input and checkpoint files
    with open(input_file, 'r', newline='', encoding='utf-8') as infile, \
         open(checkpoint_file, 'a', newline='', encoding='utf-8') as checkpointfile:

        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['osm_id']
        writer = csv.DictWriter(checkpointfile, fieldnames=fieldnames)

        # Write header if checkpoint_file is empty
        if os.stat(checkpoint_file).st_size == 0:
            writer.writeheader()

        for row in reader:
            city_state = (row['City'], row['State'])

            if city_state in city_state_osmid_mapping:
                # Re-use the cached osm_id
                row['osm_id'] = city_state_osmid_mapping[city_state]
            else:
                address = f"{row['City']}, {row['State']}, USA"

                try:
                    result = geocode_address(address)
                    if result:
                        row['osm_id'] = result
                        # Update cache
                        city_state_osmid_mapping[city_state] = result
                    else:
                        logging.warning(f"Geocoding failed for address: {address}")
                        row['osm_id'] = ''
                        city_state_osmid_mapping[city_state] = ''
                except Exception as e:
                    logging.error(f"Error geocoding address {address}: {e}")
                    row['osm_id'] = ''
                    city_state_osmid_mapping[city_state] = ''
                    continue  # Skip to the next iteration

                # Sleep to respect API rate limits
                time.sleep(sleep_interval)

            # Write the row to checkpoint_file
            writer.writerow(row)
            checkpointfile.flush()  # Ensure data is written to disk

    # Copy the checkpoint file to the final output file
    os.replace(checkpoint_file, output_file)
    logging.info(f"Geocoding completed. Results saved to {output_file}")
            
geocode_csv("fuel-prices-for-be-assessment.csv", "output.csv")


In [3]:
df1 = pd.read_csv('final.csv')
df1 = df1.dropna()
df1 = df1.drop_duplicates()
df1 = df1.reset_index(drop=True)
df1.to_csv('final.csv', index=False)
